In [ ]:
# Nötige Imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer
import tensorflow as tf
tf.keras.__version__
import torch
from transformers import pipeline
import glob
from tqdm import tqdm
import pandas as pd
import fileinput

# GPU benutzen
try:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print("Cuda available: ", torch.cuda.is_available())
    print("Cuda current device: ", torch.cuda.current_device())
    print("Cuda device s: ", torch.cuda.device(0))
    print("Cuda device count: ", torch.cuda.device_count())
    print("Cuda device name: ", torch.cuda.get_device_name(0))
    gpus = tf.config.experimental.list_physical_devices('GPU')
    memli=4000
    if gpus:
        try:
            tf.config.experimental.set_virtual_device_configuration(
            gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memli)])
            print("Memory limit for gpu is", memli,"mb")
        except RuntimeError as e:
            print(e)
except RuntimeError as e:
    print("no gpu")

In [2]:
tqdm.pandas()

In [3]:
# Tokenizer importieren, Modell erstellen
tokenizer2 =  RobertaTokenizer.from_pretrained("bhadresh-savani/roberta-base-emotion", do_lower_case=True)

# BERT-Pipeline erstellen
classifier = pipeline("text-classification",model='bhadresh-savani/roberta-base-emotion', return_all_scores=True)


In [4]:
# Funktion zur Ermittlung des Durchschnitts der Scores
def calc_scores (temp_list_pred):
    score_sadness = 0
    score_joy = 0
    score_love = 0
    score_anger = 0
    score_fear = 0
    score_suprise = 0

    for x in temp_list_pred:
        score_sadness += x[0][0]["score"]
        score_joy += x[0][1]["score"]
        score_love += x[0][2]["score"]
        score_anger += x[0][3]["score"]
        score_fear += x[0][4]["score"]
        score_suprise += x[0][5]["score"]

    score_sadness = score_sadness/le
    score_joy = score_joy/le
    score_love = score_love/le
    score_anger = score_anger/le 
    score_fear = score_fear/le
    score_suprise = score_suprise/le
    list_all = [score_sadness,score_joy,score_love,score_anger,score_fear,score_suprise]
    
    return(list_all)

### Modell anwenden

In [5]:
# Variablen erstellen
temp_list_pred=[]
list_pred = []

# Loop durch in Daten-Pipeline erstellte Exports
for files in glob.glob("rdy/16/test/*.json"):
    print("reading from: "+ files)
    # DF einlesen
    df = pd.read_json(files)
    
    # DF kürzen für Prototyp
    df = df[:5]
    
    # Loop durch DF
    for lines in df.content:
        
        # Wenn über 500 Chars, dann durchreichen in einzelnen Teilen zur Analyse-Pipeline
        if len(lines) >= 500:

            # Temporäre Variablen
            le = int(len(lines)/500)
            start = 0
            end = 500
            i = 1 

            while i <= le:
                prediction = classifier(lines[start:end],)
                temp_list_pred.append(prediction)
                start = end
                end = end + 500
                i+=1

            prediction = classifier(lines[start:len(lines)],)
            temp_list_pred.append(prediction)

        else:
            prediction = classifier(lines,)
            temp_list_pred.append(prediction)

        # Ermittlung der Score Durchschnitte
        res = calc_scores(temp_list_pred)
        list_pred.append(res)
        
    # Predictions an DF anhängen
    df["scores"] = list_pred
    
    

reading from: rdy/16/test/1_rdy.pkl


In [6]:
df

,Kategorie,Titel,Autor,Datum,content,scores
index,,,,,,
0,Local,Bubbly planet Venus starts off New Year,Blaine P. Friedlander Jr.,2012-01-01T00:12:09Z,"Set down your champagne and gaze west, as the ...","[0.015503243077546358, 0.9968882288251605, 0.0..."
1,Politics,Iowa caucuses: GOP presidential hopefuls makin...,Karen Tumulty,2012-01-01T01:55:00Z,DES MOINES — Two days before the voting begins...,"[0.03354516475902576, 1.1122658785040442, 0.01..."
2,Local,Developer of Maryland housing office in Prince...,Ovetta Wiggins,2012-01-01T00:56:45Z,A developer who stands to gain millions by bui...,"[0.09093900460616819, 1.9472337840124965, 0.03..."
3,Politics,Many Iowans still don’t know who they will cau...,Jason Horowitz,2012-01-01T01:17:00Z,"Early last week, a postcard advertising a rall...","[0.17550780467822083, 2.6054160420649817, 0.04..."
4,Local,"As homicides fall in D.C., rise in Prince Geor...",Allison Klein; Matt Zapotosky,2012-01-01T01:37:52Z,The District and Prince George’s County had ne...,"[0.19807116012088954, 2.3372668397302427, 0.04..."


0	sadness <br>
1 	joy <br>
2	love <br>
3	anger <br>
4	fear <br>
5	suprise <br>


In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)